In [1]:
# Dataset : http://cb.lk/covid_19

#!wget http://cb.lk/covid_19


--2020-08-27 10:55:12--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2020-08-27 10:55:12--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2020-08-27 10:55:13--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2020-08-27 10:55:13--  https://www.dropbox.com/s/raw/7

In [2]:
#!unzip covid_19

Archive:  covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-bbb6-ffe7

In [4]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Val"


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from  keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [6]:
# Building the Convolutional Neural Network with Keras
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape =(224,224,3)))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss = keras.losses.binary_crossentropy, optimizer='adam',metrics = ['accuracy'])



In [9]:
#print out the architechtural summary of the model
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 54, 54, 64)       

In [8]:
# Let's do the training !
train_data_gen = image.ImageDataGenerator(
    
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)



In [10]:
train_generator = train_data_gen.flow_from_directory(
    'CovidDataset/CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')




Found 224 images belonging to 2 classes.


In [11]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [12]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 60 images belonging to 2 classes.


In [16]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 7,
    epochs = 11,
    validation_data = validation_generator,
    validation_steps = 2
)

Epoch 1/11
7/7 [==============================] - 99s 14s/step - loss: 0.6852 - accuracy: 0.5000 - val_loss: 0.6797 - val_accuracy: 0.5000
Epoch 2/11
7/7 [==============================] - 87s 12s/step - loss: 0.6578 - accuracy: 0.5223 - val_loss: 0.6137 - val_accuracy: 0.7333
Epoch 3/11
7/7 [==============================] - 86s 12s/step - loss: 0.5632 - accuracy: 0.6384 - val_loss: 0.4643 - val_accuracy: 0.9500
Epoch 4/11
7/7 [==============================] - 86s 12s/step - loss: 0.5342 - accuracy: 0.7589 - val_loss: 0.3440 - val_accuracy: 0.9333
Epoch 5/11
7/7 [==============================] - 87s 12s/step - loss: 0.5752 - accuracy: 0.7321 - val_loss: 0.5668 - val_accuracy: 0.9000
Epoch 6/11
7/7 [==============================] - 86s 12s/step - loss: 0.5420 - accuracy: 0.7188 - val_loss: 0.3759 - val_accuracy: 0.9667
Epoch 7/11
7/7 [==============================] - 85s 12s/step - loss: 0.5236 - accuracy: 0.7857 - val_loss: 0.4313 - val_accuracy: 0.9667
Epoch 8/11
7/7 [===========